$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [1]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [2]:
#!pip install scikit-surprise


In [3]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [4]:
data = pd.read_csv("/content/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [5]:
utility_matrix = data.pivot(index='userID', columns='songID', values='rating')
utility_matrix.fillna(0,inplace=True)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [6]:
no_of_rows, no_of_columns = utility_matrix.shape

In [7]:
print(f'No. of Users: {no_of_rows}')
print(f'No. of Songs: {no_of_columns}')

No. of Users: 53963
No. of Songs: 56


In [8]:
U = np.random.rand(no_of_rows,3)
M = np.random.rand(3, no_of_columns)

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [9]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i,j]!=0: # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i,j] - U[i,:]@M[:,j] # Calculate the error in prediction
                    for k in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] = lr * (2*eij * M[k, j] - beta * U[i, k])
                        M[k, j] = lr * (2*eij * U[i, k] - beta * M[k, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i, j] != 0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value

                    first_part = (R[i, j] - np.dot(U[i, :], M[:, j])) ** 2 # calculate the first part of the error
                    second_part = (beta / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(M[:, j])**2) # Initialize a variable to accumelate the second part of the error

                   # for None: # Loop over each latent features dimension
                   #     second_part += None

                    e += first_part + second_part # accumelate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [10]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - R: Utility Matrix [of shape: (no. of users, no. of items)]
    - U: User Latent Features Array [of shape: (no. of users, K)]
    - M: Items Latent Features Array [of shape: (K, no. of items)]
    - epochs: No. of training epochs
    - lr: Learning rate (alpha)
    - beta: Regularization Parameter

    Function Output:
    - U: Optimized User Latent Features Array
    - M: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Create mask for non-zero entries
    mask = R != 0

    # Initialize previous error
    e_last = None

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i,j]!=0: # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i,j] - U[i,:]@M[:,j] # Calculate the error in prediction
                    for k in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] = lr * (2*eij * M[k, j] - beta * U[i, k])
                        M[k, j] = lr * (2*eij * U[i, k] - beta * M[k, j])

        ## Vectorized Error Calculation ##
        predicted = U @ M
        error_matrix = (R - predicted) * mask  # Only consider non-zero entries

        # Frobenius norm squared of the error matrix
        first_part = np.sum(error_matrix ** 2)

        # Regularization terms
        second_part = (beta / 2) * (np.linalg.norm(U, 'fro')**2 + np.linalg.norm(M, 'fro')**2)

        e = first_part + second_part

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        # Check stopping conditions
        if e_last is not None and (e < 0.001 or (e_last - e) < 0.001):
            break

        e_last = e

    return U, M

## `iii` Use the Function to to Optimize the $U$ and $V$

In [11]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=200, lr=0.01, beta=0.1)


Epoch 1/200: Total Error = 1024759.0005421746
Epoch 2/200: Total Error = 1024759.0000000054


## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [12]:
potential_items = utility_matrix.columns[utility_matrix.loc[199988] == 0]
potential_items

Index([  2263,   3785,   8063,  12709,  13859,  16548,  17029,  19670,  22763,
        24427,  25182,  28985,  36561,  40712,  42781,  42906,  43827,  45026,
        45934,  48731,  52611,  54042,  55240,  55622,  60465,  60888,  62954,
        68572,  71582,  72017,  72309,  74640,  79622,  86341,  90409,  91266,
        92881,  94535,  94604, 105421, 105433, 112023, 113954, 119103, 120147,
       122065, 123176, 125557, 126757, 131048, 132189, 134732],
      dtype='int64', name='songID')

In [13]:
predictions_matrix=U @ M
ratings=pd.DataFrame(predictions_matrix,index=utility_matrix.index, columns=utility_matrix.columns)
ratings_199988=ratings.loc[199988].sort_values(axis=0,ascending=False)
ratings_199988.head(5)

,199988
songID,
45934,3.226987e-22
113954,2.404757e-22
22763,1.608756e-22
79622,1.490579e-22
105433,6.777447e-23


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [14]:
reader = Reader()
dataset = Dataset.load_from_df(data, reader)
dataset

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [15]:
trainset, testset = train_test_split(dataset, test_size=0.2)
# Biased Model
biased_model = SVD()


In [16]:
# Non-Biased Model
non_biased_model = SVD(biased=False)


## `iii` Fit each Model on Training Data

In [17]:
# Biased Model
biased_model.fit(trainset)

In [18]:
# Non-Biased Model
non_biased_model.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [19]:
# Biased Model
biased_predictions = biased_model.test(testset)
print("Biased Model RMSE:", accuracy.rmse(biased_predictions))
print("Biased Model MAE:", accuracy.mae(biased_predictions))

RMSE: 1.5028
Biased Model RMSE: 1.502789604275752
MAE:  1.3039
Biased Model MAE: 1.3039159894296748


In [20]:
# Non-Biased Model
non_biased_predictions = non_biased_model.test(testset)
print("Non-Biased Model RMSE:", accuracy.rmse(non_biased_predictions))

RMSE: 2.0896
Non-Biased Model RMSE: 2.0895812103147318


## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [21]:
# Biased Model
songs=[]
ratings=[]
for song in potential_items:
    prediction = biased_model.predict(199988, song)
    songs.append(prediction.iid)
    ratings.append(round(prediction.est,4))
song_predictions_dict = {"recommended_song": songs,"predicted_rating": ratings}
song_predictions=pd.DataFrame(song_predictions_dict)
song_predictions.set_index("recommended_song",inplace=True)
song_predictions_sorted = song_predictions.sort_values("predicted_rating",ascending=False)
song_predictions_sorted.head(10)

,predicted_rating
recommended_song,
13859,4.7722
112023,4.7721
71582,4.6959
52611,4.3881
45026,4.3699
42781,4.3542
36561,4.2760
42906,4.1989
92881,4.1199


In [22]:
# Non-Biased Model
songs=[]
ratings=[]
for song in potential_items:
    prediction = non_biased_model.predict(199988, song)
    songs.append(prediction.iid)
    ratings.append(round(prediction.est,4))
song_predictions_dict = {"recommended_song": songs,"predicted_rating": ratings}
song_predictions=pd.DataFrame(song_predictions_dict)
song_predictions.set_index("recommended_song",inplace=True)
song_predictions_sorted = song_predictions.sort_values("predicted_rating",ascending=False)
song_predictions_sorted.head(10)

,predicted_rating
recommended_song,
94604,3.8267
134732,3.7613
122065,3.5661
13859,3.5386
48731,3.1146
55240,2.9070
40712,2.6119
119103,2.4882
43827,2.4822


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$